### Import helper Libraries and data

In [1]:
import torch
import torch.nn as nn
from torch import tensor
from torch import Tensor
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dataLoader import Dataloader

/home/doit/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [4]:
dataset_train = Dataloader(['fold1_20', 'fold2_20', 'fold3_20', 'fold4_20', 'fold5_20',\
                             'fold6_20', 'fold7_20', 'fold8_20', 'fold9_20'], test = False)
dataset_test = Dataloader(['fold10_20'], test = True)

train_dataloader = torch.utils.data.DataLoader(dataset_train, batch_size = 10)
test_dataloader = torch.utils.data.DataLoader(dataset_test, batch_size = 10)

train_inputs, train_featres = next(iter(train_dataloader))
test_inputs, test_featres = next(iter(test_dataloader))

/home/doit/anaconda3/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/home/doit/anaconda3/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore


In [5]:
def conv(ni, nf, ks=5, pool=True):
    res = nn.Conv2d(ni, nf, kernel_size=ks, stride = 1 )
    if pool: res = nn.Sequential(res, nn.MaxPool2d((4, 2)), nn.ReLU())
    else: res = nn.Sequential(res, nn.ReLU())
    return res

### Change device to CUDA

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

### Create Model

In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.relu_stack = nn.Sequential(
            conv(1, 24, ks=5, pool=True), 
            conv(24, 48, ks=5, pool=True),  
            conv(48, 48, ks=5, pool=False),
            nn.Flatten(),
            nn.Linear(2400, 64),
            nn.Dropout(p=0.5),
            nn.Linear(64, 10),
            nn.Dropout(p=0.5)
                                )

    def forward(self, x):
        logits = self.relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [34]:
model

NeuralNetwork(
  (relu_stack): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 24, kernel_size=(5, 5), stride=(1, 1))
      (1): MaxPool2d(kernel_size=(4, 2), stride=(4, 2), padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(24, 48, kernel_size=(5, 5), stride=(1, 1))
      (1): MaxPool2d(kernel_size=(4, 2), stride=(4, 2), padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(48, 48, kernel_size=(5, 5), stride=(1, 1))
      (1): ReLU()
    )
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=2400, out_features=64, bias=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
    (7): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
from torchsummary import summary
summary(model, input_size=(1, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 124, 124]             624
         MaxPool2d-2           [-1, 24, 31, 62]               0
              ReLU-3           [-1, 24, 31, 62]               0
            Conv2d-4           [-1, 48, 27, 58]          28,848
         MaxPool2d-5            [-1, 48, 6, 29]               0
              ReLU-6            [-1, 48, 6, 29]               0
            Conv2d-7            [-1, 48, 2, 25]          57,648
              ReLU-8            [-1, 48, 2, 25]               0
           Flatten-9                 [-1, 2400]               0
           Linear-10                   [-1, 64]         153,664
          Dropout-11                   [-1, 64]               0
           Linear-12                   [-1, 10]             650
          Dropout-13                   [-1, 10]               0
Total params: 241,434
Trainable params:

### Train and Test

In [36]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

epochs = 50
for t in range(epochs):
    if (t+1) % 10 == 0:
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 10
-------------------------------
loss: 2.168492  [    0/  180]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.240807 

Epoch 20
-------------------------------
loss: 2.153559  [    0/  180]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.266800 

Epoch 30
-------------------------------
loss: 2.228149  [    0/  180]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.268005 

Epoch 40
-------------------------------
loss: 1.724818  [    0/  180]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.264922 

Epoch 50
-------------------------------
loss: 1.711749  [    0/  180]
Test Error: 
 Accuracy: 5.0%, Avg loss: 0.282199 

Done!


### Check the total number of parameters

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
params

In [ ]:
for i in model.parameters():
    print(i.shape)

In [ ]:
p = sum([np.prod(p.size()) for p in model.parameters()])
p

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

In [ ]:
# If you want to calculate only the trainable parameters
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

### Save the model